# Implementing our novel approach with Naive Bayes

> Preprocessing: Clean and preprocess your dataset. This may include handling missing values, encoding categorical variables, and scaling features.

## Load dataset

In [194]:
from libs import data
from libs import kde_lib
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from libs.exp_lib import Density_model
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin


X0, y0 = data.load_data("banana")

X_train, X_test, y_train, y_test = train_test_split(X0, y0, test_size=0.3, random_state=42)

/Users/biad/Desktop/THESIS/Tests/RKDE_HHO/libs/../data/banana/banana
Loaded banana data: 5300 samples, 2 dimensions, 2 labels
classes = {1.0, -1.0}


In [195]:
""" # Get the dimension of our data
dim = X_train.ndim

# Calculate the number of classes and features
classes = np.unique(y_train)
n_features = X_train.shape[1]

# Initialize robust densities
robust_densities = np.zeros((len(X_train), dim))

print(robust_densities.shape)
kernel = 'gaussian'
#h = .5 """


" # Get the dimension of our data\ndim = X_train.ndim\n\n# Calculate the number of classes and features\nclasses = np.unique(y_train)\nn_features = X_train.shape[1]\n\n# Initialize robust densities\nrobust_densities = np.zeros((len(X_train), dim))\n\nprint(robust_densities.shape)\nkernel = 'gaussian'\n#h = .5 "

## Step2 - step3 : Robust Kernel Density Estimation (RKDE) & Bandwidth Selection using HHO:

- Implement the RKDE algorithm with IRLS and Robust M-estimation (Hampel function) for each class.
- Calculate robust densities for each class based on the RKDE algorithm.


In [196]:

""" for class_label in classes:
    # GET for each class
    class_indices = np.where(y_train==class_label)[0]
    class_X = X_train[class_indices]
    h = kde_lib.hho_bandwith_selection(class_X,class_X)
    #h = .5
    # Get for each feature
    rkde_feature =[]
    for d in range(dim):
        X = class_X[:,d] 
        X = X[:,np.newaxis]
        #print(X.shape)
        model = Density_model("rkde", "banana", 0,kernel,h)
        model.fit(X,X,grid=None)
        rkde = model.density
        
        robust_densities[class_indices,d] = rkde[:,0] """

    


' for class_label in classes:\n    # GET for each class\n    class_indices = np.where(y_train==class_label)[0]\n    class_X = X_train[class_indices]\n    h = kde_lib.hho_bandwith_selection(class_X,class_X)\n    #h = .5\n    # Get for each feature\n    rkde_feature =[]\n    for d in range(dim):\n        X = class_X[:,d] \n        X = X[:,np.newaxis]\n        #print(X.shape)\n        model = Density_model("rkde", "banana", 0,kernel,h)\n        model.fit(X,X,grid=None)\n        rkde = model.density\n        \n        robust_densities[class_indices,d] = rkde[:,0] '

In [197]:
print(robust_densities[:10])

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [198]:
print(robust_densities.shape)

(3710, 2)


## Step4: Incorporating RKDE into Naive Bayes:

> - Modify Naive Bayes classifier to use the RKDE densities instead of traditional Gaussian densities.
> - For prediction, calculate the likelihood using the robust densities obtained from RKDE.

In [199]:
class RobustNaiveBayes(BaseEstimator, ClassifierMixin):

    def __init__(self) -> None:
        self.class_priors = None
        self.robust_densities = None
        self.kernel = 'gaussian'

    
    def fit(self, X, y):
        """
        Fit the robust Naive Bayes model with RKDE densities.

        Parameters:
        X (array-like): Training data features.
        y (array-like): Training data labels.
        """
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        # Calculate class priors
        self.class_priors = np.array([np.mean(y==c) for c in self.classes])

        # Initialize robust densities
        robust_densities = np.zeros((n_samples, n_features))

        for class_label in self.classes :
            # GET for each class
            class_indices = np.where(y==class_label)[0]
            class_data = X[class_indices]
            #bandwidth = kde_lib.hho_bandwith_selection(class_data,class_data)
            bandwidth = .5
            # Get for each feature
            for feature in range(n_features):
                feature_data = class_data[:,feature]
                feature_data = feature_data[:,np.newaxis]
                model = Density_model("rkde","banana",0,self.kernel,bandwidth)
                model.fit(feature_data, feature_data, grid=None)
                rkde = model.density

                robust_densities[class_indices,feature] = rkde[:,0]
        self.robust_densities = robust_densities

        

    def predict(self, X):
        """
        Predict class labels and RKDE likelihoods for input data.

        Parameters:
        X (array-like): Input data features.

        Returns:
        y_pred (array-like): Predicted class labels.
        rkde_likelihoods (array-like): RKDE likelihoods for each class.
        """
        pass

In [200]:
# Create and fit the RobustNaiveBayes classifier
classifier = RobustNaiveBayes()
classifier.fit(X_train, y_train)

Stop at 7 iterations
Stop at 100 iterations
Stop at 7 iterations
Stop at 100 iterations
Stop at 6 iterations
Stop at 100 iterations
Stop at 7 iterations
Stop at 100 iterations


In [204]:
# Check the shape of our train data and the robust densities
print(X_train.shape)
print(classifier.robust_densities.shape)

(3710, 2)
(3710, 2)
